Entrenamiento de un modelo Random Forest para clasificación de triaje NEWS
--------------------------------------------------------------------------
- Lee el dataset limpio con saturación imputada (CSV)
- Prepara variables y etiqueta (KTAS_expert → Emergencia/No‑Emergencia)
- Entrena un RandomForestClassifier dentro de un Pipeline que incluye imputación
- Evalúa métrica principal AUROC y guarda el modelo como artefacto .joblib
- Pensado para ejecutarse en la capa gratuita de AWS (t2.micro ó local)

In [23]:
from pathlib import Path
import joblib
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier

In [60]:
# ---------------------------------------------------------------------------
# Configuración
# ---------------------------------------------------------------------------
DATA_PATH = Path("triageDataSet_final_sintetico.csv")  # Ruta al CSV 
MODEL_DIR = Path("model_artifact")             # Carpeta de salida
MODEL_DIR.mkdir(exist_ok=True)
MODEL_PATH = MODEL_DIR / "news_rf_pipeline.joblib"
RANDOM_STATE = 42
TEST_SIZE = 0.20

In [61]:
# ---------------------------------------------------------------------------
# 1. Carga de datos
# ---------------------------------------------------------------------------
print("[INFO] Cargando dataset limpio…")
df = pd.read_csv(DATA_PATH)

[INFO] Cargando dataset limpio…


In [62]:
# ---------------------------------------------------------------------------
# 2. Ingeniería de la etiqueta
#    KTAS_expert: 1‑3 = Emergencia (1), 4‑5 = No‑emergencia (0)
# ---------------------------------------------------------------------------
print("[INFO] Transformando etiqueta KTAS_expert…")
if "KTAS_expert" not in df.columns:
    raise KeyError("La columna 'KTAS_expert' no existe en el dataset.")

df["label"] = df["KTAS_expert"]


[INFO] Transformando etiqueta KTAS_expert…


In [63]:
# ---------------------------------------------------------------------------
# 3. Selección de variables predictoras
# ---------------------------------------------------------------------------
FEATURES = [
  "Sex", "Age", "Mental", "NRS_pain", "SBP", "DBP", "HR", "RR", "BT", "Saturation"
]
X = df[FEATURES]
y = df["label"]

In [64]:
# ---------------------------------------------------------------------------
# 4. Conjunto de entrenamiento y validación
# ---------------------------------------------------------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=TEST_SIZE, stratify=y, random_state=RANDOM_STATE
)


In [65]:
# ---------------------------------------------------------------------------
# 5. Pipeline de pre‑procesado + modelo
#    - Imputación mediana para cualquier NaN residual
# ---------------------------------------------------------------------------
print("[INFO] Construyendo pipeline…")

numeric_features = FEATURES  # todas son numéricas tras codificar Sex
preprocessor = ColumnTransformer(
    transformers=[
        ("num", SimpleImputer(strategy="median"), numeric_features)
    ],
    remainder="drop",
)

clf = RandomForestClassifier(
    n_estimators=1000,
    max_depth=50,
    min_samples_split=5,
    min_samples_leaf=1,
    class_weight="balanced",
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

pipeline = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("rf", clf),
    ]
)

[INFO] Construyendo pipeline…


In [66]:
# ---------------------------------------------------------------------------
# 6. Entrenamiento
# ---------------------------------------------------------------------------
print("[INFO] Entrenando modelo…")
pipeline.fit(X_train, y_train)

[INFO] Entrenando modelo…


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['Sex', 'Age', 'Mental',
                                                   'NRS_pain', 'SBP', 'DBP',
                                                   'HR', 'RR', 'BT',
                                                   'Saturation'])])),
                ('rf',
                 RandomForestClassifier(class_weight='balanced', max_depth=50,
                                        min_samples_split=5, n_estimators=1000,
                                        n_jobs=-1, random_state=42))])

In [67]:
# ---------------------------------------------------------------------------
# 7. Evaluación
# ---------------------------------------------------------------------------
print("[INFO] Evaluando en conjunto de validación…")
proba_val = pipeline.predict_proba(X_val)[:, 1]
pred_val = pipeline.predict(X_val)
#auc = roc_auc_score(y_val, proba_val)
#print(f"AUROC: {auc:.3f}")
print(classification_report(y_val, pred_val, digits=3))

[INFO] Evaluando en conjunto de validación…
              precision    recall  f1-score   support

           1      0.939     0.949     0.944        98
           2      0.691     0.670     0.681        97
           3      0.476     0.515     0.495        97
           4      0.545     0.429     0.480        98
           5      0.795     0.918     0.852        97

    accuracy                          0.696       487
   macro avg      0.689     0.696     0.690       487
weighted avg      0.690     0.696     0.690       487



In [68]:
# ---------------------------------------------------------------------------
# 8. Persistencia del modelo
# ---------------------------------------------------------------------------
print(f"[INFO] Guardando modelo en {MODEL_PATH}…")
joblib.dump(pipeline, MODEL_PATH)
print("[INFO] Modelo guardado correctamente ✔️")

[INFO] Guardando modelo en model_artifact\news_rf_pipeline.joblib…
[INFO] Modelo guardado correctamente ✔️


# ---------------------------------------------------------------------------
# Notas finales
# ---------------------------------------------------------------------------
El archivo news_rf_pipeline.joblib incluye toda la lógica (imputación + modelo).

Para cargarlo en una API Flask/FastAPI o Lambda:

      model = joblib.load('news_rf_pipeline.joblib')
      proba = model.predict_proba(new_patient_df)[:, 1]

Manténer el mismo orden y nombres de columnas.